### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import datetime

### Inspect Data

In [2]:
#create pandas dataframe from tabular policing data
policing_df = pd.read_csv('denver-1.nov.csv')
policing_df.head()

,raw_row_number,date,time,location,lat,lng,district,precinct,type,disposition,arrest_made,citation_issued,warning_issued,outcome
0,1,2012-11-13,00:02:22,8500 W CRESTLINE AVE,39.618329,-105.092691,4,423,vehicular,K - Street Check Completed,False,False,False,NaN
1,16,2012-11-29,00:25:01,8500 W CRESTLINE AVE,39.618329,-105.092691,4,423,vehicular,Party Advised,False,False,False,NaN
2,72,2012-11-23,22:58:10,4830 S HOLLAND WAY,39.630666,-105.102551,4,423,vehicular,K - Street Check Completed,False,False,False,NaN
3,121,2012-11-27,17:48:33,4885 S QUEBEC ST,39.626174,-104.904062,3,324,pedestrian,In Service,False,False,False,NaN
4,138,2012-11-14,23:46:58,W LAYTON AVE / S WADSWORTH BLVD,39.630150,-105.081693,4,423,vehicular,Party Advised,False,False,False,NaN


In [3]:
#remove unwanted info and check shape of dataframe
policing_df = policing_df.drop(['lat', 'lng', 'raw_row_number'], axis = 1) 
policing_df.shape

(6365, 11)

### Filtering and Aggregation

In [4]:
#create new dataframe that shows policing outcomes and total stops by district
by_district = policing_df.groupby("district").sum()
by_district["counts"] = policing_df["district"].value_counts()
by_district

,arrest_made,citation_issued,warning_issued,counts
district,,,,
1,200,207,141,1158
2,254,209,155,1338
3,113,256,151,968
4,149,181,138,1158
5,60,111,67,571
6,186,84,57,1004
7,0,1,0,2
None,23,16,23,166


In [5]:
#SANITY CHECK
#just want to see if my df above is accurate
dis4 = policing_df[policing_df['district'] == '4']
dis4['arrest_made'].value_counts()
#149 arrests, 1158 total stops. We're good.

False    1009
True      149
Name: arrest_made, dtype: int64

In [6]:
#create column for calculated arrest% and warning % statistic
by_district['arrest%'] = by_district['arrest_made']/by_district['counts']
by_district['warning%'] = by_district['warning_issued']/by_district['counts']
by_district

,arrest_made,citation_issued,warning_issued,counts,arrest%,warning%
district,,,,,,
1,200,207,141,1158,0.172712,0.121762
2,254,209,155,1338,0.189836,0.115845
3,113,256,151,968,0.116736,0.155992
4,149,181,138,1158,0.128670,0.119171
5,60,111,67,571,0.105079,0.117338
6,186,84,57,1004,0.185259,0.056773
7,0,1,0,2,0.000000,0.000000
None,23,16,23,166,0.138554,0.138554


### Convert to json

In [7]:
#create json data from pandas df
police_json = by_district.to_json()
police_json

'{"arrest_made":{"1":200,"2":254,"3":113,"4":149,"5":60,"6":186,"7":0,"None":23},"citation_issued":{"1":207,"2":209,"3":256,"4":181,"5":111,"6":84,"7":1,"None":16},"warning_issued":{"1":141,"2":155,"3":151,"4":138,"5":67,"6":57,"7":0,"None":23},"counts":{"1":1158,"2":1338,"3":968,"4":1158,"5":571,"6":1004,"7":2,"None":166},"arrest%":{"1":0.1727115717,"2":0.1898355755,"3":0.1167355372,"4":0.1286701209,"5":0.1050788091,"6":0.1852589641,"7":0.0,"None":0.1385542169},"warning%":{"1":0.121761658,"2":0.1158445441,"3":0.1559917355,"4":0.1191709845,"5":0.1173380035,"6":0.0567729084,"7":0.0,"None":0.1385542169}}'